Documentation: https://www.tensorflow.org/tensorboard/dataframe_api.

Make sure:
1. Logs are uploaded: `tensorboard dev upload --logdir models/text/bert-base-uncased/fake_real/logs --name {name}`
2. You know the experiment ID, in url. 

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import tensorboard as tb

: 

In [ ]:
pd

In [4]:
import code.loader
#loader.load_data

ModuleNotFoundError: No module named 'code.loader'; 'code' is not a package

In [9]:
experiment_id = "uRVZx3rvQdq00w4cNeHJvw"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
df = experiment.get_scalars()
df

,run,tag,step,value
0,.,eval/accuracy,3,1.000000e+00
1,.,eval/loss,3,2.026134e-01
2,.,eval/runtime,3,7.600000e+00
3,.,eval/samples_per_second,3,1.053000e+00
4,.,eval/steps_per_second,3,1.320000e-01
5,.,train/epoch,3,1.000000e+00
6,.,train/total_flos,3,4.391693e+12
7,.,train/train_loss,3,2.662190e-01
8,.,train/train_runtime,3,1.020272e+02
9,.,train/train_samples_per_second,3,1.670000e-01


In [11]:
dfw = experiment.get_scalars(pivot=True)
dfw

,run,step,eval/accuracy,eval/loss,eval/runtime,eval/samples_per_second,eval/steps_per_second,train/epoch,train/total_flos,train/train_loss,train/train_runtime,train/train_samples_per_second,train/train_steps_per_second
0,.,3,1.0,0.202613,7.6,1.053,0.132,1.0,4.391693e+12,0.266219,102.027199,0.167,0.029


In [14]:
csv_path = './tmp/tb_experiment_1.csv'
dfw.to_csv(csv_path, index=False)
dfw_roundtrip = pd.read_csv(csv_path)
pd.testing.assert_frame_equal(dfw_roundtrip, dfw)

In [16]:
# Filter the DataFrame to only validation data, which is what the subsequent
# analyses and visualization will be focused on.
dfw_validation = dfw[dfw.run.str.endswith("/validation")]
# Get the optimizer value for each row of the validation DataFrame.
optimizer_validation = dfw_validation.run.apply(lambda run: run.split(",")[0])

plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
sns.lineplot(data=dfw_validation, x="step", y="epoch_accuracy",
             hue=optimizer_validation).set_title("accuracy")
plt.subplot(1, 2, 2)
sns.lineplot(data=dfw_validation, x="step", y="epoch_loss",
             hue=optimizer_validation).set_title("loss")

In [ ]:
adam_min_val_loss = dfw_validation.loc[optimizer_validation=="adam", :].groupby(
    "run", as_index=False).agg({"epoch_loss": "min"})
rmsprop_min_val_loss = dfw_validation.loc[optimizer_validation=="rmsprop", :].groupby(
    "run", as_index=False).agg({"epoch_loss": "min"})
sgd_min_val_loss = dfw_validation.loc[optimizer_validation=="sgd", :].groupby(
    "run", as_index=False).agg({"epoch_loss": "min"})
min_val_loss = pd.concat([adam_min_val_loss, rmsprop_min_val_loss, sgd_min_val_loss])

sns.boxplot(data=min_val_loss, y="epoch_loss",
            x=min_val_loss.run.apply(lambda run: run.split(",")[0]))

In [ ]:
# Perform pairwise comparisons between the minimum validation losses
# from the three optimizers.
_, p_adam_vs_rmsprop = stats.ttest_ind(
    adam_min_val_loss["epoch_loss"],
    rmsprop_min_val_loss["epoch_loss"]) 
_, p_adam_vs_sgd = stats.ttest_ind(
    adam_min_val_loss["epoch_loss"],
    sgd_min_val_loss["epoch_loss"]) 
_, p_rmsprop_vs_sgd = stats.ttest_ind(
    rmsprop_min_val_loss["epoch_loss"],
    sgd_min_val_loss["epoch_loss"]) 
print("adam vs. rmsprop: p = %.4f" % p_adam_vs_rmsprop)
print("adam vs. sgd: p = %.4f" % p_adam_vs_sgd)
print("rmsprop vs. sgd: p = %.4f" % p_rmsprop_vs_sgd)